In [1]:
import pickle as pkl
import datetime

import csv
# Scraping items
from bs4 import BeautifulSoup
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os

from IPython.core.display import display, HTML
import concurrent.futures

from collections import Counter
import numpy as np
from scipy import stats

import pandas as pd
from subprocess import call
import concurrent.futures
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#from pymongo import MongoClient

In [2]:
#Path on Ubuntu
# chromedriver = "/home/williamcottrell72/Downloads/chromedriver_linux64/chromedriver" # path to the chromedriver executable
#Path on Mac
chromedriver="/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [4]:
# List of reviews
with open('SF_Urls','r') as f:
    urls=pd.read_csv(f)
urls2=[x.strip().strip('\'').strip('\'') for x in list(urls.columns)]

In [10]:
web_pages=["https://www.tripadvisor.com/Attractions-g60713-Activities-San_Francisco_California.html#FILTERED_LIST"]
for k in range(30,750,30):
    url="https://www.tripadvisor.com/Attractions-g60713-Activities-oa"+str(k)+"-San_Francisco_California.html#FILTERED_LIST"
    web_pages.append(url)

In [11]:
web_pages[-1]

'https://www.tripadvisor.com/Attractions-g60713-Activities-oa720-San_Francisco_California.html#FILTERED_LIST'

In [27]:
SF_Urls=[]
for w in web_pages[:3]:
    driver.get(w)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    listing=soup.find_all('div', class_='listing_info')[0]
    href=listing.find_all(class_='listing_title')[0].find('a')['href']
    url="https://www.tripadvisor.com/"+href
    SF_Urls.append(url)

In [31]:
SF_Urls[1]

'https://www.tripadvisor.com//Attraction_Review-g60713-d105418-Reviews-San_Francisco_Museum_of_Modern_Art_SFMOMA-San_Francisco_California.html'

In [13]:
soup=BeautifulSoup(driver.page_source,'html.parser')

In [17]:
listing=soup.find_all('div', class_='listing_info')[0]

In [26]:
listing.find_all(class_='listing_title')[0].find('a')['href']

'/Attraction_Review-g60713-d104943-Reviews-Yerba_Buena_Gardens-San_Francisco_California.html'

In [34]:
sf_attraction_template='https://www.tripadvisor.com/Attractions-g60713-Activities-oa{}-San_Francisco_California.html#FILTERED_LIST'


def make_url_list(sf_attraction_template):
    
    #The inital and other pages are slightly different so we just break these up
    
    initial_page=sf_attraction_template.format('')
    other_pages=[sf_attraction_template.format(30*i) for i in range(1,3)]
    all_pages=[initial_page]+other_pages
    
    urls_final=[]
    
    for p in all_pages:
        
        try:
            driver.get(p)
            soup=BeautifulSoup(driver.page_source,'html.parser')
            individual_urls=soup.find_all('div', class_='listing_title')

            for iu in individual_urls:
                partial_url=iu.find('a')['href']
                full_url='https://www.tripadvisor.com'+partial_url
                urls_final.append(full_url)
        except:
            pass
        
    return urls_final

In [35]:
urls_final=make_url_list(sf_attraction_template)

In [33]:
#urls_final=make_url_list(sf_attraction_template)

# with open('urls_final.pkl','wb') as f:
#     pkl.dump(urls_final,f)

with open('urls_final.pkl','rb') as f:
    urls_final=pkl.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'urls_final.pkl'

In [7]:
driver.get(urls_final[30])

In [36]:
location_info2={}
for url in urls_final[:4]:
    driver.get(url)
    soup=BeautifulSoup(driver.page_source,'html.parser')
    try:
        rating=soup.find('span',class_='ui_bubble_rating bubble_40')['alt'][0]
    except:
        rating='None'
    try:
        tags=[x.text for x in soup.find('div',class_='detail').find_all('a',href=True)]
    except:
        tags=['None']
    name=soup.find(id='HEADING').text
    location_info2[name]=[rating,tags]

    


In [37]:
location_info2

{'Alcatraz Island': ['None',
  ['Nature & Parks',
   'Sights & Landmarks',
   'Nature & Wildlife Areas',
   'National Parks',
   'Islands',
   'Historic Sites']],
 'Golden Gate Bridge': ['None', ['Sights & Landmarks', 'Bridges']],
 'Cable Cars': ['None', ['Transportation', 'Tramways']],
 'Lombard Street': ['None',
  ['Sights & Landmarks', 'Points of Interest & Landmarks']]}

In [26]:
# with open('location_info.pkl','wb') as f:
#      pkl.dump(location_info,f)

with open('location_info2.pkl','rb') as f:
    location_info2 = pkl.load(f)

In [34]:
loc_info={x.strip('\n').strip('\u200e') : location_info2[x] for x in list(location_info2.keys())}

In [35]:
with open('good_data/loc_info.pkl','wb') as f:
    pkl.dump(loc_info,f)

In [ ]:
new_keys

In [19]:
Counter(tags+['Neighborhoods'])

Counter({'Sights & Landmarks': 1, 'Piers & Boardwalks': 1, 'Neighborhoods': 2})

In [ ]:
#Need map taking places to counts.
